In [240]:
import os
import mmap
import random
import pandas as pd

In [222]:
import torch
import torch.nn.functional as F
import torch.nn as nn

device = 'mps' if torch.backends.mps.is_built() else 'cpu'
print(f"Current Device: {device}")

Current Device: mps


In [231]:
block_size = 64
batch_size = 128
max_iters = 3000
eval_iters = 100
learning_rate = 3e-4
n_emb = 384
n_layer = 8
n_head = 8
dropout = 0.2

In [232]:
chars = ""
with open("vocab.txt", 'r', encoding='utf-8') as f:
        text = f.read()
        chars = sorted(list(set(text)))

vocab_size = len(chars)
print(f"Vocab: {chars} \nSize: {vocab_size}")

string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for ch, i in string_to_int.items()}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: "".join([int_to_string[i] for i in l])

assert decode(encode(df.heavy.iloc[0])) == df.heavy.iloc[0]

Vocab: ['\n', '#', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'] 
Size: 22


In [233]:
# memory map for using small snippets of text from a single file of any size
def get_random_chunk(split):
    filename = "data/train.txt" if split == 'train' else "data/val.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            
            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
            
    return data


def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y
            

In [234]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [235]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.query = nn.Linear(n_emb, head_size, bias=False)
        self.key = nn.Linear(n_emb, head_size, bias=False)
        self.value = nn.Linear(n_emb, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        q = self.query(x) # B, T, hs
        k = self.key(x)   # B, T, hs
        w = q @ k.transpose(-2, -1) * (k.shape[-1] ** -0.5) # B, T, T
        w = w.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # B, T, T
        w = F.softmax(w, dim=-1) # B, T, T
        w = self.dropout(w)
        v = self.value(x) # B, T, hs
        out = w @ v # B, T, T @ B, T, hs -> B, T, hs
        return out


class MultiHeadAttention(nn.Module):
    def __init__(self, n_head, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(n_head*head_size, n_emb)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # B, T, C
        out = self.dropout(self.proj(out))
        return out


class FeedForward(nn.Module):
    def __init__(self, n_emb):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_emb, 4*n_emb),
            nn.ReLU(),
            nn.Linear(4*n_emb, n_emb),
            nn.Dropout(dropout))

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    def __init__(self, n_emb, n_head):
        super().__init__()
        head_size = n_emb // n_head
        self.mha = MultiHeadAttention(n_head, head_size)
        self.ln1 = nn.LayerNorm(n_emb)
        self.ffw = FeedForward(n_emb)
        self.ln2 = nn.LayerNorm(n_emb)
    
    def forward(self, x):
        y = self.mha(x) # B, T, C
        x = self.ln1(x + y)
        y = self.ffw(x)
        x = self.ln2(x + y)
        return x


class GPTLM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_emb_table = nn.Embedding(vocab_size, n_emb)
        self.pos_emb_table = nn.Embedding(block_size, n_emb)
        self.blocks = nn.Sequential(*[Block(n_emb, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_emb) # B, T, C -> B, 1, 1(mean or var) -> B, T, C
        self.lm_head = nn.Linear(n_emb, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        

    def forward(self, index, targets=None):
        # index, targets are both (B, T) tensor of integers
        B, T = index.shape
        token_emb = self.token_emb_table(index) # B, T, C
        pos_emb = self.pos_emb_table(torch.arange(T).to(device)) # T, C
        x = token_emb + pos_emb # B, T, C
        x = self.blocks(x) # B, T, C
        x = self.ln_f(x) # B, T, C
        logits = self.lm_head(x) # B, T, Vocab_size

        if targets is None:
            loss = None
        else:
            logits = logits.view(B*T, vocab_size)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            index_cond = index[:, -block_size:]
            logits, _ = model(index_cond)
            probs = F.softmax(logits[:, -1, :], dim=-1) # B, C
            index_next = torch.multinomial(probs, num_samples=1) # B, 1
            index = torch.cat([index, index_next], dim=1) # B, T+1
        return index

In [236]:
model = GPTLM(vocab_size).to(device)

In [237]:
model

GPTLM(
  (token_emb_table): Embedding(22, 384)
  (pos_emb_table): Embedding(64, 384)
  (blocks): Sequential(
    (0): Block(
      (mha): MultiHeadAttention(
        (heads): ModuleList(
          (0-7): 8 x Head(
            (query): Linear(in_features=384, out_features=48, bias=False)
            (key): Linear(in_features=384, out_features=48, bias=False)
            (value): Linear(in_features=384, out_features=48, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (ffw): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln2): LayerNorm

In [238]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        out = estimate_loss()
        print(f"Step: {iter} Train Loss {out['train']:.4f} Val Loss {out['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

Step: 0 Train Loss 3.1703 Val Loss 3.1712
Step: 100 Train Loss 1.8705 Val Loss 1.8192
Step: 200 Train Loss 1.2164 Val Loss 1.1577
Step: 300 Train Loss 1.0037 Val Loss 0.9487
Step: 400 Train Loss 0.8905 Val Loss 0.8332
Step: 500 Train Loss 0.7840 Val Loss 0.7499
Step: 600 Train Loss 0.7481 Val Loss 0.6745
Step: 700 Train Loss 0.7054 Val Loss 0.6359
Step: 800 Train Loss 0.6692 Val Loss 0.5962
Step: 900 Train Loss 0.6310 Val Loss 0.5785
Step: 1000 Train Loss 0.6054 Val Loss 0.5625
Step: 1100 Train Loss 0.5767 Val Loss 0.5272
Step: 1200 Train Loss 0.5855 Val Loss 0.5179
Step: 1300 Train Loss 0.5718 Val Loss 0.4970
Step: 1400 Train Loss 0.5702 Val Loss 0.4930
Step: 1500 Train Loss 0.5539 Val Loss 0.4761
Step: 1600 Train Loss 0.5388 Val Loss 0.4796
Step: 1700 Train Loss 0.5295 Val Loss 0.4337
Step: 1800 Train Loss 0.5136 Val Loss 0.4439
Step: 1900 Train Loss 0.4618 Val Loss 0.4418
Step: 2000 Train Loss 0.5050 Val Loss 0.4590
Step: 2100 Train Loss 0.4845 Val Loss 0.4473
Step: 2200 Train Loss 

In [241]:

checkpoint_dir = "checkpoints"
checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_{iter}.pt")
torch.save({
    'iter': iter,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': out
}, checkpoint_path)

In [243]:
checkpoint_path = f"{checkpoint_dir}/checkpoint_2999.pt"
checkpoint = torch.load(checkpoint_path)

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
iter = checkpoint['iter']
loss = checkpoint['loss']
print(f"Loading checkpoints on step {iter} sucussessfully, loss is {loss}")

Loading checkpoints on step 2999 sucussessfully, loss is {'train': tensor(0.4751), 'val': tensor(0.3901)}


In [247]:
class BOT(GPTLM):
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            index_cond = index[:, -block_size:]
            logits, _ = model(index_cond)
            probs = F.softmax(logits[:, -1, :], dim=-1) # B, C
            index_next = torch.multinomial(probs, num_samples=1) # B, 1
            index = torch.cat([index, index_next], dim=1) # B, T+1
        return index

In [249]:
bot = BOT(vocab_size)

In [250]:
prompt = 'QVQLKESGPGLVQPSETLSLTCTVSGFSLTSYSVSWVRQPSGKGPEWMGRMW'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_seq = decode(bot.generate(context.unsqueeze(0), max_new_tokens=50)[0].tolist())
generated_seq

'QVQLKESGPGLVQPSETLSLTCTVSGFSLTSYSVSWVRQPSGKGPEWMGRMWYDDGDTAYNSALKSRLSISRDTSKNQVFLKMNSLQTEDTGTYYCARHTHR'